## Chapter 4: Model-Free Prediction


Compared to **Dynamics Programming**, **Model-free prediction** algorithms aim to estimate the **value function of a certain policy** *without knowing the MDP*. Mainly, **Monte-Carlo Prediction** and **Temporal-Difference learning (TD-learning)** are known as a **model-free** prediction algorithms. 

### Monte-Carlo Prediction

According to Wikipedia, **Monte-Carlo (MC)** method is a broad class of algorithms that repeated random sampling to obtain numerical results. For example, the value of $\pi$ can be approximated using a Monte Carlo method, by uniformly scattering number of points over the square. 

In Reinforcement Learning, it can be used to evaulate the value function of the policy $\pi$ by sampling return of multiple episodes.
</br>
</br>
<font size="4">
$$\begin{align}
v_\pi(s) = \mathbb{E}_\pi[G_t |S_t = s]
\end{align}$$
</font>
where return $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots \gamma^{T-1} R_T$. 

Monte-Carlo policy evaluation consider empirical mean return as value. 




### Temporal-Difference Learning

**Temporal-Difference (TD)** learning is a 